In [23]:
import numpy as np
import pandas as pd
import datetime as dt
import pymysql
pymysql.install_as_MySQLdb()

from sqlalchemy import create_engine
from config import dbuser, dbpasswd, dburi, dbport, dbname

### Extract CSVs into DataFrames

In [24]:
# read food_data.csv into dataframe
ffile = "../raw_data/food_data.csv"
# define lambda function to parse date column
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m')
fdf = pd.read_csv(ffile, parse_dates=['yearmonth'], date_parser=dateparse)
fdf.head()

,year,month,state,genus_species,serotype_or_genotype,etiology_status,location_of_preparation,illnesses,hospitalizations,deaths,food_vehicle,contaminated_ingredient,yearmonth
0,2009,1,Minnesota,Norovirus,NaN,Suspected,Restaurant - Sit-down dining,2,0.0,0.0,NaN,NaN,2009-01-01
1,2009,1,Minnesota,Norovirus,NaN,Confirmed,NaN,16,0.0,0.0,NaN,NaN,2009-01-01
2,2009,1,Minnesota,Norovirus,NaN,Suspected,Restaurant - Sit-down dining,5,0.0,0.0,NaN,NaN,2009-01-01
3,2009,1,Minnesota,Norovirus,NaN,Confirmed,"Restaurant - ""Fast-food""(drive up service or p...",3,0.0,0.0,NaN,NaN,2009-01-01
4,2009,1,Minnesota,Norovirus,NaN,Confirmed,Restaurant - other or unknown type,21,0.0,0.0,cookies,NaN,2009-01-01


In [25]:
# check datatypes for columns
fdf.dtypes

year                                int64
month                               int64
state                              object
genus_species                      object
serotype_or_genotype               object
etiology_status                    object
location_of_preparation            object
illnesses                           int64
hospitalizations                  float64
deaths                            float64
food_vehicle                       object
contaminated_ingredient            object
yearmonth                  datetime64[ns]
dtype: object

In [26]:
# read temperature.csv into dataframe
tfile = "../raw_data/temperature.csv"
# define lambda function to parse date column
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')

tdf = pd.read_csv(tfile, parse_dates=['dt'], date_parser=dateparse)
tdf.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
0,1855-05-01,25.544,1.171,Acre,Brazil
1,1855-06-01,24.228,1.103,Acre,Brazil
2,1855-07-01,24.371,1.044,Acre,Brazil
3,1855-08-01,25.427,1.073,Acre,Brazil
4,1855-09-01,25.675,1.014,Acre,Brazil


In [27]:
tdf.dtypes

dt                               datetime64[ns]
AverageTemperature                      float64
AverageTemperatureUncertainty           float64
State                                    object
Country                                  object
dtype: object

### Transform food_data DataFrame

In [28]:
# keep data only until 2013 
fdf = fdf[fdf['yearmonth'] < '2014-01']
fdf.year.unique()

array([2009, 2010, 2011, 2012, 2013, 2005, 2004, 1999, 1998, 2000, 2001,
       2002, 2003, 2006, 2007, 2008])

In [29]:
# Create a filtered dataframe from specific columns
fcols = ['yearmonth', 'state', 'genus_species', 
         'etiology_status', 'location_of_preparation', 'illnesses',
         'hospitalizations', 'deaths', 'food_vehicle']
ftrans = fdf[fcols].copy()

# Reset index
ftrans = ftrans.reset_index(drop="True")

ftrans.head()

,yearmonth,state,genus_species,etiology_status,location_of_preparation,illnesses,hospitalizations,deaths,food_vehicle
0,2009-01-01,Minnesota,Norovirus,Suspected,Restaurant - Sit-down dining,2,0.0,0.0,NaN
1,2009-01-01,Minnesota,Norovirus,Confirmed,NaN,16,0.0,0.0,NaN
2,2009-01-01,Minnesota,Norovirus,Suspected,Restaurant - Sit-down dining,5,0.0,0.0,NaN
3,2009-01-01,Minnesota,Norovirus,Confirmed,"Restaurant - ""Fast-food""(drive up service or p...",3,0.0,0.0,NaN
4,2009-01-01,Minnesota,Norovirus,Confirmed,Restaurant - other or unknown type,21,0.0,0.0,cookies


### Transform temperature DataFrame

In [30]:
# only keep data for US for 1998 to 2013
tdf = tdf[tdf.Country == 'United States']
tdf = tdf[(tdf['dt'] >= '1998-01-01') & (tdf['dt'] < '2014-01-01')]
tdf.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
10508,1998-01-01,9.114,0.188,Alabama,United States
10509,1998-02-01,9.828,0.147,Alabama,United States
10510,1998-03-01,12.261,0.187,Alabama,United States
10511,1998-04-01,16.744,0.165,Alabama,United States
10512,1998-05-01,23.599,0.121,Alabama,United States


In [31]:
tdf.tail()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
626435,2013-05-01,10.607,0.208,Wyoming,United States
626436,2013-06-01,16.267,0.276,Wyoming,United States
626437,2013-07-01,20.222,0.133,Wyoming,United States
626438,2013-08-01,19.621,0.217,Wyoming,United States
626439,2013-09-01,15.811,1.101,Wyoming,United States


In [32]:
# Create a filtered dataframe from specific columns
tcols = ["dt", "AverageTemperature", "State"]
ttrans = tdf[tcols].copy()

# Rename the column headers
ttranscols = ["yearmonth", "average_temp", "state"]
rendict = dict(zip(tcols, ttranscols))
ttrans = ttrans.rename(columns=rendict)

# Reset index
ttrans = ttrans.reset_index(drop="True")

ttrans.head()

,yearmonth,average_temp,state
0,1998-01-01,9.114,Alabama
1,1998-02-01,9.828,Alabama
2,1998-03-01,12.261,Alabama
3,1998-04-01,16.744,Alabama
4,1998-05-01,23.599,Alabama


In [33]:
ttrans.shape

(9639, 3)

In [34]:
ftrans.shape

(17350, 9)

### Create database connection

In [35]:
cstring = f'{dbuser}:<wini>@{dburi}:{dbport}/{dbname}'
engine = create_engine(f'mysql://{cstring}')

In [36]:
# Confirm tables
engine.table_names()

['food_data', 'temperature']

### Load DataFrames into database

In [37]:
ftrans.to_sql(name='food_data', con=engine, if_exists='append', index=True, index_label='id')

In [38]:
ttrans.to_sql(name='temperature', con=engine, if_exists='append', index=True, index_label='id')